In [12]:
import plotly.graph_objects as go

values = [
    ['test accuracy', '5-fold accuracy', 'lr', 'fov', 'chs', 'epochs'], #1st col
    [5, 3, 2, 0],
    [],
    [],
]


fig = go.Figure(data=[go.Table(
  columnorder = [1,2,3,4],
  columnwidth = [200,200],
  header = dict(
    values = [[],['<b>GAT</b>'],['<b>GCN<b>'],['<b>Cheb<b>']],
    line_color='darkslategray',
    fill_color='royalblue',
    align=['left','center'],
    font=dict(color='white', size=12),
    height=40
  ),
  cells=dict(
    values=values,
    line_color='darkslategray',
    fill=dict(color=['royalblue', 'white']),
    font=dict(color=['white', 'black']),
    align=['left', 'center'],
    font_size=12,
    height=30)
    )
])
fig.show()

Find all the kfold accuracy, split data depending on model

In [19]:
import sys 
sys.path.append('../')

import pickle 
import numpy as np
import pandas as pd
import utils

data   = {'mean_accuracy':[], 'model':[]}
setups = []

with open('fixedGraphResults.pickle', 'rb') as handle:
    temp = pickle.load(handle)
    for t in temp:
        t['model'] = t['model'].__name__+'1'
    setups += temp

with open('functionalGraphResults.pickle', 'rb') as handle:
    temp = pickle.load(handle)
    for t in temp:
        t['model'] = t['model'].__name__+'2'
    setups += temp

with open('BasicResults.pickle', 'rb') as handle:
    temp = pickle.load(handle)
    for t in temp:
        t['model'] = t['model'].__name__
    setups += temp

for s in setups:
    mean_accuracy = np.mean(s['val_acc'])
    data['mean_accuracy'].append(mean_accuracy)
    data['model'].append(s['model'])

df = pd.DataFrame(data)
maximums = df.groupby('model')['mean_accuracy'].max()

import plotly.express as px 

fig = px.bar(maximums, y="mean_accuracy", x=maximums.index)
fig.show()

In [36]:
model_type = 'Cheb2'
best = {'val_acc':0}
for s in setups:
    if s['model'] != model_type:
        continue
    current_best = np.mean(best['val_acc'])
    mean_accuracy = np.mean(s['val_acc'])
    if mean_accuracy > current_best:
        best = s 

print(best)
print(np.mean(best['val_acc']))

{'model': 'Cheb2', 'lr': 0.001, 'fov': 3, 'chs': 32, 'epochs': 20, 'val_loss': [0.33841687025648826, 0.27882149946153834, 0.2829737347833226, 0.30291937690102655, 0.3080981884779555], 'val_acc': [0.8561797752808988, 0.8741573033707866, 0.8831460674157303, 0.8719101123595505, 0.8808988764044944]}
0.8732584269662921


In [84]:
import plotly.graph_objs as go

bests = {
    'LIN':{'val_acc':0},
    'MLP':{'val_acc':0},
    'CNN':{'val_acc':0},
    'GAT1':{'val_acc':0},
    'GAT2':{'val_acc':0},
    'GCN1':{'val_acc':0},
    'GCN2':{'val_acc':0},
    'Cheb1':{'val_acc':0},
    'Cheb2':{'val_acc':0},
}

for s in setups:
    model = s['model']
    current_best = np.mean(bests[model]['val_acc'])
    mean_accuracy = np.mean(s['val_acc'])
    if mean_accuracy > current_best:
        bests[model] = s

x = list(bests.keys())
y = np.array([bests[model]['val_acc'] for model in x]).flatten()
x = np.repeat(x, 5)
trace = go.Scatter(x=x, y=y, mode='markers')

layout = go.Layout(
    plot_bgcolor='white',
)


fig = go.Figure(trace, layout)
fig.update_xaxes(dict(showgrid=False, linewidth=0.2, linecolor='black', ticks='outside', color='black', title_text='models'),)
fig.update_yaxes(dict(showline=True, linewidth=0.2, linecolor='black', title_text='binary cross entropy loss'))
fig.show()


In [59]:
x = list(bests.keys())
print(x)

['LIN', 'MLP', 'CNN', 'GAT1', 'GAT2', 'GCN1', 'GCN2', 'Cheb1', 'Cheb2']
